### 0. Initial Setting

In [1]:
# %%capture
# !pip install datasets==1.0.2
# !pip install transformers==4.2.1

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

cache_dir = "/data4/yoomcache"
model_cache_dir = os.path.join(cache_dir, 'huggingface')
data_cache_dir = os.path.join(cache_dir, 'datasets')
checkpoint_dir = os.path.join(cache_dir, 'checkpoint')

import logging
logging.getLogger().setLevel(logging.CRITICAL)
logging.basicConfig(level=logging.INFO)


import torch
from datasets import load_dataset, load_metric, load_from_disk
from transformers import BertTokenizer, RobertaTokenizer, GPT2Tokenizer
from transformers import AutoConfig, EncoderDecoderConfig, EncoderDecoderModel
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

import wandb
wandb.init(project="testing-gptONLY", entity="yoom-private")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yoom-private (use `wandb login --relogin` to force relogin)


### 1. Initialize Model

In [3]:
config_encoder = AutoConfig.from_pretrained("gpt2", cache_dir=model_cache_dir)
config_decoder = AutoConfig.from_pretrained("gpt2", cache_dir=model_cache_dir)
config = EncoderDecoderConfig.from_encoder_decoder_configs(config_encoder, config_decoder, cache_dir=model_cache_dir)
model = EncoderDecoderModel(config=config)
# model.save_pretrained("roberta2gpt", cache_dir=model_cache_dir)
# model = EncoderDecoderModel.from_pretrained("roberta2gpt", cache_dir=model_cache_dir)

model.encoder.h = model.encoder.h[:4]
model.decoder.transformer.h = model.decoder.transformer.h[-4:]

In [4]:
model.decoder

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (crossattention): GPT2Attention(
          (c_attn): Conv1D()
          (q_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_cross_attn): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
      

In [7]:
# make sure GPT2 appends EOS in begin and end
def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
    outputs = [self.bos_token_id] + token_ids_0 + [self.eos_token_id]
    return outputs

GPT2Tokenizer.build_inputs_with_special_tokens = build_inputs_with_special_tokens


encoder_tokenizer = GPT2Tokenizer.from_pretrained("gpt2", cache_dir=model_cache_dir)
encoder_tokenizer.pad_token = encoder_tokenizer.unk_token
# encoder_tokenizer.bos_token = encoder_tokenizer.cls_token  # CLS token will work as BOS token
# encoder_tokenizer.eos_token = encoder_tokenizer.sep_token  # SEP token will work as EOS token

decoder_tokenizer = GPT2Tokenizer.from_pretrained("gpt2", cache_dir=model_cache_dir)
# set pad_token_id to unk_token_id -> be careful here as unk_token_id == eos_token_id == bos_token_id
decoder_tokenizer.pad_token = decoder_tokenizer.unk_token


model.config.decoder_start_token_id = encoder_tokenizer.cls_token_id
model.config.eos_token_id = encoder_tokenizer.sep_token_id
model.config.pad_token_id = encoder_tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size


# set decoding params
model.config.decoder_start_token_id = decoder_tokenizer.bos_token_id
model.config.eos_token_id = decoder_tokenizer.eos_token_id
model.config.max_length = 142
model.config.min_length = 56
model.config.no_repeat_ngram_size = 3
model.early_stopping = True
model.length_penalty = 2.0
model.num_beams = 4

In [8]:
# Freeze decoder parameters
for param in model.decoder.parameters():
    param.requires_grad = False


### 2. Preparing Dataset

In [9]:
# map data correctly
def map_to_encoder_decoder_inputs(batch):    # Tokenizer will automatically set [BOS] <text> [EOS] 
    encoder_length, decoder_length = 512, 128
    inputs = encoder_tokenizer(batch["article"], 
                               padding="max_length", 
                               truncation=True, 
                               max_length=encoder_length)
    outputs = decoder_tokenizer(batch["highlights"], 
                                padding="max_length", 
                                truncation=True, 
                                max_length=decoder_length)
    
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["decoder_input_ids"] = outputs.input_ids
    batch["labels"] = outputs.input_ids.copy()
    batch["decoder_attention_mask"] = outputs.attention_mask

    # complicated list comprehension here because pad_token_id alone is not good enough to know whether label should be excluded or not
    batch["labels"] = -100 if batch["decoder_attention_mask"] == 0 else batch["labels"]

    assert len(inputs.input_ids) == encoder_length
    assert len(outputs.input_ids) == decoder_length

    return batch

In [10]:
if os.path.exists(os.path.join(cache_dir, 'preprocessed/train')):
    train_dataset = load_from_disk(os.path.join(cache_dir, 'preprocessed/train'))
else:
    train_dataset = load_dataset("ccdv/cnn_dailymail", "3.0.0", split="train", cache_dir=data_cache_dir)
    train_dataset = train_dataset.map(
        map_to_encoder_decoder_inputs, 
        # batched=True, 
        # batch_size=batch_size, 
        remove_columns=['id', 'article', 'highlights'],
    )
    train_dataset.set_format(
        type="torch", 
        columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
    )
    
    train_dataset.save_to_disk(os.path.join(cache_dir, 'preprocessed/train'))


if os.path.exists(os.path.join(cache_dir, 'preprocessed/val')):
    val_dataset = load_from_disk(os.path.join(cache_dir, 'preprocessed/val'))
else:
    val_dataset = load_dataset("ccdv/cnn_dailymail", "3.0.0", split="validation", cache_dir=data_cache_dir)
    val_dataset = val_dataset.map(
        map_to_encoder_decoder_inputs, 
        # batched=True, 
        # batch_size=batch_size, 
        remove_columns=['id', 'article', 'highlights'],
    )
    val_dataset.set_format(
        type="torch", 
        columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
    )
    val_dataset.save_to_disk(os.path.join(cache_dir, 'preprocessed/val'))

### Training Model

In [11]:
# load rouge for validation
rouge = load_metric("rouge")
# rouge = load_metric("rouge", experiment_id=1)

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # all unnecessary tokens are removed
    pred_str = decoder_tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids == -100] = decoder_tokenizer.eos_token_id
    label_str = decoder_tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4),
    }

In [ ]:
batch_size = 16

# set training arguments - these params are not really tuned, feel free to change
training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    output_dir=os.path.join(checkpoint_dir, "gptONLY"),
    do_train=True,
    do_eval=True,
    # do_predict=True,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=1e-4, 
#     weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0,
    num_train_epochs=100,
    max_steps=-1,
    lr_scheduler_type='cosine',
    
    logging_strategy='steps',
    save_strategy='steps',
    evaluation_strategy='steps',
    logging_steps=1000,
    save_steps=1000,
    eval_steps=1000,
    warmup_steps=10000,
    save_total_limit=10,
    overwrite_output_dir=True,
)

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# start training
trainer.train()

/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 28711
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 179500
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You ma

Step,Training Loss,Validation Loss,Rouge2 Precision,Rouge2 Recall,Rouge2 Fmeasure
1000,5.023300,3.589779,0.000000,0.000000,0.000000
2000,4.471300,3.203995,0.000000,0.000000,0.000000
3000,4.194800,3.085265,0.000000,0.000000,0.000000
4000,4.072400,3.006499,0.000000,0.000000,0.000000
5000,3.970900,2.895184,0.000200,0.000100,0.000200
6000,3.810200,2.781841,0.000100,0.000100,0.000100
7000,3.667800,2.682241,0.000000,0.000100,0.000000
8000,3.577300,2.575866,0.000800,0.001600,0.001000
9000,3.420300,2.447604,0.001500,0.004000,0.002100
10000,3.301900,2.363680,0.001500,0.003900,0.002100


***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint/gptONLY/checkpoint-1000
Configuration saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-1000/config.json
Model weights saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-1000/pytorch_model.bin
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/

Configuration saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-10000/config.json
Model weights saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-10000/pytorch_model.bin
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint/gptONLY/checkpoint-11000
Configuration saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-11000/config.json
Model weights 

/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint/gptONLY/checkpoint-19000
Configuration saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-19000/config.json
Model weights saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-19000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint/gptONLY/checkpoint-9000] due to args.save_total

...

/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint/gptONLY/checkpoint-135000
Configuration saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-135000/config.json
Model weights saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-135000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint/gptONLY/checkpoint-125000] due to args.save_

***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint/gptONLY/checkpoint-143000
Configuration saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-143000/config.json
Model weights saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-143000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint/gptONLY/checkpoint-133000] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, Fut

Configuration saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-151000/config.json
Model weights saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-151000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint/gptONLY/checkpoint-141000] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint/gptONLY/checkp

Deleting older checkpoint [/data4/yoomcache/checkpoint/gptONLY/checkpoint-149000] due to args.save_total_limit
/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint/gptONLY/checkpoint-160000
Configuration saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-160000/config.json
Model weights saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-160000/pytorc

/home/yoomin/anaconda3/envs/py38/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:531: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)
***** Running Evaluation *****
  Num examples = 1337
  Batch size = 16
Saving model checkpoint to /data4/yoomcache/checkpoint/gptONLY/checkpoint-168000
Configuration saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-168000/config.json
Model weights saved in /data4/yoomcache/checkpoint/gptONLY/checkpoint-168000/pytorch_model.bin
Deleting older checkpoint [/data4/yoomcache/checkpoint/gptONLY/checkpoint-158000] due to args.save_